# Generación de Dataset <Período, Customer, Producto>

Vale la pena predecir aquellos clientes que compraron una sola vez? No en los ultimos 3 meses, sino mas anteriormente. 
* Rta: No borra nada!!!!

Como podemos saber si un producto esta discontinuado?
* Cuando dejaron de haber ventas del mismo para siempre.
* Estamos hablando se sell-in, que son las ventas de La Multinacional a sus clientes (cadenas de supermercados, mayoristas, distribuidores). No hablamos del sell-out que son las ventas a consumidor final.
* La Multinacional da la bienvenida a tu idea de incorporar esos factores externos y que les expliques como afectan a sus ventas




Ayer me quedó una duda respecto a la generación de filas en cero para sell in. Entiendo que es conveniente generar registros con valor cero para evitar "huecos" en las combinaciones de customer_id y product_id durante los períodos en que ambos están activos. Mi pregunta es: <i> Si un customer_id nunca compró un determinado producto, ¿igualmente conviene generar registros en cero para todos los períodos en los que ese cliente estuvo activo? Y, en caso de que así sea, ¿aunque el producto haya tenido ventas solo en un período y el cliente también estuvo activo en ese mismo período, igual se beneficia el análisis al agregar ese registro en cero? </i>
* Si un customer_id nunca compró un determinado producto, ¿igualmente conviene generar registros en cero para todos los períodos en los que ese cliente estuvo activo? 
 * * Afirmativo
* Y, en caso de que así sea, ¿aunque el producto haya tenido ventas solo en un período y el cliente también estuvo activo en ese mismo período, igual se beneficia el análisis al agregar ese registro en cero?
* * Afirmativo

Si se decide utilizar solamente las 800 series de tiempo en forma individual,  ya sea para un algoritmo de la Estadistica Clasica como ARIMA  o uno de Deep Learning ,  en realidad es hacer un loop que genere 800 modelos. Ahi la informacion extra no cuenta.

Si se decide utilzar un algoritmo como LightGBM es necesaria  crear un dataset estructurado a partir de las series de tiempo, y cada registro sera un   < periodo, customer_id, product_id,   multiples_campos_historicos_derivados,  informacion_externa>.   En este caso,  cada registro cuenta,  cada bit de informacion va a ayudar al LightGBM a predecir mejor, incluso un  cliente que vivio un solo mes y decidio no comprar un producto ese unico mes de efimera vida.  Ese registro contribuira a la "probabilidad de compra" al  "monto promedio comprado"  al considerarse con los datos de otros clientes ya establecidos

* No le quites posibilidad de aprender al algortimo  cercenando el dataset !
* No le quites datos outliers
* No le quites el mes malo de 2019-08  si  existe la ingeniosa posibilidad de marcarlo como "problematico"
* Se ingenioso en como incorporar a medias y lags a las ventas de un mes problematico,  aunque ello demande crear mas campos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Levantamos el dataset base

In [2]:
# 3. MERGE INICIAL
df = pd.read_csv("../data/preprocessed/base.csv", sep=",")
print(f"Ventas-Productos-Stocks: {df.shape[0]} filas y {df.shape[1]} columnas")

Ventas-Productos-Stocks: 2945818 filas y 13 columnas


# Iván

In [ ]:
import pandas as pd
import gc

sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')

df = pd.merge(sellin, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])
print(f"Ventas-Productos-Stocks: {df.shape[0]} filas y {df.shape[1]} columnas")
del sellin, productos, stocks

df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")

periodos = pd.date_range(start=df['periodo_dt'].min(), end=df['periodo_dt'].max(), freq="MS")
productos = df['product_id'].unique()
clientes = df['customer_id'].unique()

idx = pd.MultiIndex.from_product([productos, clientes, periodos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)
completo["periodo"] = completo["periodo"].dt.strftime("%Y%m").astype(int)

del periodos, productos, clientes
gc.collect()


Ventas-Productos-Stocks: 2945818 filas y 13 columnas


14

In [ ]:
completo["periodo_dt"] = pd.to_datetime(completo["periodo"].astype(str), format="%Y%m")

Eliminar productos que no nacieron

In [ ]:
nacimiento_producto = df.groupby("product_id")["periodo_dt"].agg(["min"]).reset_index()
# Renombrar columna max a muerte_cliente_dt
nacimiento_producto = nacimiento_producto.rename(columns={'min': 'nacimiento_producto'})


# Unir con df_final para traer fecha de muerte del cliente
completo = completo.merge(nacimiento_producto, on='product_id', how='left')

# Filtrar filas donde periodo_dt > muerte_cliente_dt
completo = completo[completo['periodo_dt'] >= completo['nacimiento_producto']]

# Opcional: eliminar columna auxiliar
# df_final = df_final.drop(columns=['muerte_cliente_dt'])

print(f"✅ Dataset filtrado con {len(completo):,} filas.")


✅ Dataset filtrado con 21,425,136 filas.


In [ ]:
del nacimiento_producto
gc.collect()

0

Eliminar productos que murieron

In [ ]:
muerte_de_producto = df.groupby("product_id")["periodo_dt"].agg(["max"]).reset_index()
# Renombrar columna max a muerte_cliente_dt
muerte_de_producto = muerte_de_producto.rename(columns={'max': 'muerte_de_producto'})


# Unir con df_final para traer fecha de muerte del cliente
completo = completo.merge(muerte_de_producto, on='product_id', how='left')

# Filtrar filas donde periodo_dt > muerte_cliente_dt
completo = completo[completo['periodo_dt'] <= completo['muerte_de_producto']]

# Opcional: eliminar columna auxiliar
# df_final = df_final.drop(columns=['muerte_cliente_dt'])

print(f"✅ Dataset filtrado con {len(completo):,} filas.")


✅ Dataset filtrado con 18,818,634 filas.


In [ ]:
del muerte_de_producto
gc.collect()

0

Eliminamos clientes que murieron: por ahora no aplicarlo, porque no tendria sentido.

In [ ]:
# df_final["periodo_dt"] = pd.to_datetime(df_final["periodo"].astype(str), format="%Y%m")

muerte_cliente = df.groupby("customer_id")["periodo_dt"].agg(["max"]).reset_index()

# Renombrar columna max a muerte_cliente_dt
muerte_cliente = muerte_cliente.rename(columns={'max': 'muerte_cliente_dt'})

# Unir con df_final para traer fecha de muerte del cliente
completo = completo.merge(muerte_cliente, on='customer_id', how='left')

# Filtrar filas donde periodo_dt > muerte_cliente_dt
completo = completo[completo['periodo_dt'] <= completo['muerte_cliente_dt']]

# Opcional: eliminar columna auxiliar
# df_final = df_final.drop(columns=['muerte_cliente_dt'])

print(f"✅ Dataset filtrado con {len(completo):,} filas.")


✅ Dataset filtrado con 17,208,299 filas.


In [ ]:
del muerte_cliente
gc.collect()

0

Eliminamos clientes que no nacieron

In [ ]:
# df_final["periodo_dt"] = pd.to_datetime(df_final["periodo"].astype(str), format="%Y%m")

nacimiento_cliente = df.groupby("customer_id")["periodo_dt"].agg(["min"]).reset_index()

# Renombrar columna max a nacimiento_cliente_dt
nacimiento_cliente = nacimiento_cliente.rename(columns={'min': 'nacimiento_cliente_dt'})

# Unir con df_final para traer fecha de muerte del cliente
completo = completo.merge(nacimiento_cliente, on='customer_id', how='left')

# Filtrar filas donde periodo_dt > nacimiento_cliente_dt
completo = completo[completo['periodo_dt'] >= completo['nacimiento_cliente_dt']]

# Opcional: eliminar columna auxiliar
# df_final = df_final.drop(columns=['nacimiento_cliente_dt'])

print(f"✅ Dataset filtrado con {len(completo):,} filas.")


✅ Dataset filtrado con 17,173,448 filas.


In [ ]:
del nacimiento_cliente
gc.collect()

0

In [ ]:
mask = completo['periodo'] == 201912
completo[mask]['product_id'].nunique()

927

Cruzamos con productos

In [ ]:
productos = pd.read_csv("../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
completo = completo.merge(productos, how='left', on="product_id")
del productos
gc.collect()

0

Cruzamos con stocks

In [ ]:
stocks = pd.read_csv("../data/raw/tb_stocks.csv", sep='\t')
stocks = stocks.groupby(by=["periodo", "product_id"]).agg({"stock_final": "sum"}).reset_index()
completo = completo.merge(stocks, how='left', on=['periodo', 'product_id'])
del stocks
gc.collect()
completo.head()

,product_id,customer_id,periodo,periodo_dt,nacimiento_producto,muerte_de_producto,nacimiento_cliente_dt,cat1,cat2,cat3,brand,sku_size,stock_final
0,20524,10234,201701,2017-01-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN
1,20524,10234,201702,2017-02-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN
2,20524,10234,201703,2017-03-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN
3,20524,10234,201704,2017-04-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN
4,20524,10234,201705,2017-05-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN


Cruzamos con ventas

In [ ]:
sellin = pd.read_csv("../data/raw/sell-in.csv", sep='\t')
# Agrupar ventas por periodo, cliente y producto
dt = sellin.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
                                                                "cust_request_tn":"sum",
                                                                "cust_request_qty":"sum",
                                                                "plan_precios_cuidados":"first"
                                                                }).reset_index()
df_completo = completo.merge(dt, how='left', on=['periodo', 'customer_id','product_id'])
df_completo['tn'] = df_completo['tn'].fillna(0)
del sellin, dt, completo
gc.collect()

9

In [ ]:
df_completo.head()

,product_id,customer_id,periodo,periodo_dt,nacimiento_producto,muerte_de_producto,nacimiento_cliente_dt,cat1,cat2,cat3,brand,sku_size,stock_final,tn,cust_request_tn,cust_request_qty,plan_precios_cuidados
0,20524,10234,201701,2017-01-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.05300,0.05300,2.0,0.0
1,20524,10234,201702,2017-02-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN
2,20524,10234,201703,2017-03-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.01514,0.01514,1.0,0.0
3,20524,10234,201704,2017-04-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN
4,20524,10234,201705,2017-05-01,2017-01-01,2019-12-01,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN


In [ ]:
# 7. Unir con datos reales
df_completo["nacimiento_producto"] = df_completo["nacimiento_producto"].dt.strftime("%Y%m").astype(int)
df_completo["muerte_de_producto"] = df_completo["muerte_de_producto"].dt.strftime("%Y%m").astype(int)
df_completo.head()

,product_id,customer_id,periodo,periodo_dt,nacimiento_producto,muerte_de_producto,nacimiento_cliente_dt,cat1,cat2,cat3,brand,sku_size,stock_final,tn,cust_request_tn,cust_request_qty,plan_precios_cuidados
0,20524,10234,201701,2017-01-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.05300,0.05300,2.0,0.0
1,20524,10234,201702,2017-02-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN
2,20524,10234,201703,2017-03-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.01514,0.01514,1.0,0.0
3,20524,10234,201704,2017-04-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN
4,20524,10234,201705,2017-05-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN


Creamos el target

In [ ]:
# Asegurarte de tener 'periodo_dt' (datetime) en completo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear DataFrame auxiliar con tn como target y fecha adelantada
ventas_futuras = df_completo[['periodo_dt', 'customer_id', 'product_id', 'tn']].copy()
ventas_futuras['periodo_target_dt'] = ventas_futuras['periodo_dt'] - pd.DateOffset(months=2)
ventas_futuras = ventas_futuras.rename(columns={'tn': 'target'})

# Merge con completo usando periodo adelantado
df_completo = df_completo.merge(
    ventas_futuras[['periodo_target_dt', 'customer_id', 'product_id', 'target']],
    how='left',
    left_on=['periodo_dt', 'customer_id', 'product_id'],
    right_on=['periodo_target_dt', 'customer_id', 'product_id']
)

# Eliminar columna auxiliar
df_completo = df_completo.drop(columns=['periodo_target_dt'])

print(f"✅ Target generado. Filas con target no nulo: {df_completo['target'].notna().sum()}")


In [ ]:
df_completo.head(13)

,product_id,customer_id,periodo,periodo_dt,nacimiento_producto,muerte_de_producto,nacimiento_cliente_dt,cat1,cat2,cat3,brand,sku_size,stock_final,tn,cust_request_tn,cust_request_qty,plan_precios_cuidados,target
0,20524,10234,201701,2017-01-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.05300,0.05300,2.0,0.0,0.01514
1,20524,10234,201702,2017-02-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN,0.00000
2,20524,10234,201703,2017-03-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.01514,0.01514,1.0,0.0,0.00000
3,20524,10234,201704,2017-04-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN,0.03786
4,20524,10234,201705,2017-05-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN,0.00000
5,20524,10234,201706,2017-06-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.03786,0.03786,1.0,0.0,0.00000
6,20524,10234,201707,2017-07-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN,0.00000
7,20524,10234,201708,2017-08-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN,0.00000
8,20524,10234,201709,2017-09-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN,0.07571
9,20524,10234,201710,2017-10-01,201701,201912,2017-01-01,HC,VAJILLA,Cristalino,Importado,500.0,NaN,0.00000,NaN,NaN,NaN,0.00000


Verifico las NaN en el target: Existen porque hay clientes que solo compraron 2 veces.

In [ ]:
nan_count = df_completo['target'].isna().sum()
print(f"🔍 Total de NaN en target: {nan_count}")

🔍 Total de NaN en target: 1431761


In [ ]:
mask = (df_completo['periodo'] < 201911) & (df_completo['target'].isna())
df_completo[mask]

,product_id,customer_id,periodo,periodo_dt,nacimiento_producto,muerte_de_producto,nacimiento_cliente_dt,cat1,cat2,cat3,brand,sku_size,stock_final,tn,cust_request_tn,cust_request_qty,plan_precios_cuidados,target
312896,21151,10234,201805,2018-05-01,201701,201806,2017-01-01,FOODS,SOPAS Y CALDOS,Salsas Dry,MAGGI,10.0,NaN,0.0,NaN,NaN,NaN,NaN
312897,21151,10234,201806,2018-06-01,201701,201806,2017-01-01,FOODS,SOPAS Y CALDOS,Salsas Dry,MAGGI,10.0,NaN,0.0,NaN,NaN,NaN,NaN
312914,21151,10032,201805,2018-05-01,201701,201806,2017-01-01,FOODS,SOPAS Y CALDOS,Salsas Dry,MAGGI,10.0,NaN,0.0,NaN,NaN,NaN,NaN
312915,21151,10032,201806,2018-06-01,201701,201806,2017-01-01,FOODS,SOPAS Y CALDOS,Salsas Dry,MAGGI,10.0,NaN,0.0,NaN,NaN,NaN,NaN
312932,21151,10217,201805,2018-05-01,201701,201806,2017-01-01,FOODS,SOPAS Y CALDOS,Salsas Dry,MAGGI,10.0,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17118772,21216,10504,201910,2019-10-01,201910,201911,2019-10-01,HC,PROFESIONAL,SUAVIZANTE,VIVERE,2000.0,7.99999,0.0,NaN,NaN,NaN,NaN
17118774,21216,10455,201910,2019-10-01,201910,201911,2019-10-01,HC,PROFESIONAL,SUAVIZANTE,VIVERE,2000.0,7.99999,0.0,NaN,NaN,NaN,NaN
17118776,21216,10479,201910,2019-10-01,201910,201911,2019-10-01,HC,PROFESIONAL,SUAVIZANTE,VIVERE,2000.0,7.99999,0.0,NaN,NaN,NaN,NaN
17118778,21216,10538,201910,2019-10-01,201910,201911,2019-10-01,HC,PROFESIONAL,SUAVIZANTE,VIVERE,2000.0,7.99999,0.0,NaN,NaN,NaN,NaN


Guardamos el dataset

In [ ]:
df_completo.drop(columns=['periodo_dt'], inplace=True)
df_completo.to_csv("../data/preprocessed/dataset.csv", index=False, sep=',')

# Fran

In [ ]:
import pandas as pd
import gc

sellin = pd.read_csv("../data/raw/sell-in.csv", sep='\t')
productos = pd.read_csv("../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
stocks = pd.read_csv("../data/raw/tb_stocks.csv", sep='\t')

df = pd.merge(sellin, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])
del sellin, productos, stocks

# Agrupar ventas por periodo, cliente y producto
dt = df.groupby(by=["periodo","customer_id","product_id"]).agg({"tn":"sum",
                                                                "cust_request_tn":"sum",
                                                                "cust_request_qty":"sum",
                                                                "cat1":"first",
                                                                "cat2":"first",
                                                                "cat3":"first",
                                                                "brand":"first",
                                                                "sku_size":"first",
                                                                "stock_final":"first",
                                                                "plan_precios_cuidados":"first"
                                                                }).reset_index()
# 1. Calcular primeros períodos

periodo_producto =  df.groupby(['periodo', 'product_id'])["tn"].sum().rename('periodo_producto')
nacimiento_producto = df.groupby('product_id')['periodo'].min().rename('nacimiento_producto')
muerte_cliente = df.groupby('customer_id')['periodo'].max().rename('muerte_cliente')

# 2. Crear todas las combinaciones posibles
productos = df['product_id'].unique()
clientes = df['customer_id'].unique()
periodos = df['periodo'].unique()

# 3. Crear matriz de combinaciones válidas usando broadcasting
idx = pd.MultiIndex.from_product([productos, clientes, periodos], names=['product_id', 'customer_id', 'periodo'])
completo = idx.to_frame(index=False)
# 4 filtrar combinaciones periodo_producto
completo = completo.merge(periodo_producto, on=['periodo', 'product_id'], how='left')
completo = completo[completo['periodo_producto'].notna()]
# 5 Filtrar combinaciones nacimiento_producto
completo = completo.merge(nacimiento_producto, on='product_id', how='left')
completo = completo[completo['periodo'] >= completo['nacimiento_producto']]
completo = completo[completo['nacimiento_producto'] < 201910]

# 6. Filtrar combinaciones muerte_cliente
completo = completo.merge(muerte_cliente, on='customer_id', how='left')
completo = completo[completo['periodo'] <= completo['muerte_cliente']]


# 7. Unir con datos reales
df_completo = completo.merge(dt, how='left')
df_completo['tn'] = df_completo['tn'].fillna(0)

# Convertir periodo a datetime con día fijo
df_completo['periodo_dt'] = pd.to_datetime(df_completo['periodo'], format='%Y%m')

# Crear columna periodo_target_dt desplazada 2 meses
df_completo['periodo_target_dt'] = df_completo['periodo_dt'] + pd.DateOffset(months=2)

# DataFrame auxiliar con ventas futuras
ventas_futuras = df_completo[['periodo_dt', 'customer_id', 'product_id', 'tn']].copy()
ventas_futuras = ventas_futuras.rename(columns={
    'periodo_dt': 'periodo_target_dt',
    'tn': 'target'
})

# Hacemos el merge usando fechas datetime directamente
dt = pd.merge(
    df_completo,
    ventas_futuras,
    how='left',
    on=['periodo_target_dt', 'customer_id', 'product_id']
)
dt = dt.drop(columns=['periodo_target_dt']) # Ya usamos esta para hacer el merge
del df_completo, ventas_futuras, completo, idx, periodo_producto, nacimiento_producto, muerte_cliente
gc.collect()

dt.shape

(16998193, 18)

# Ivan Viejo

In [ ]:
# Supongamos que df ya contiene las columnas: periodo, customer_id, product_id, tn
df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")

# Paso 1: Rango total de periodos
todos_los_periodos = pd.date_range(start=df["periodo_dt"].min(), end=df["periodo_dt"].max(), freq="MS")

# Paso 2: Todos los clientes únicos
todos_los_clientes = df["customer_id"].unique()

# Paso 3: Determinar vida útil de cada producto
vida_producto = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()

# Paso 4: Generar combinaciones (periodo, producto) considerando restricciones
combinaciones_producto_periodo = []
fecha_limite_nuevos = pd.to_datetime("2020-03", format="%Y-%m")



for _, row in vida_producto.iterrows():
    producto = row["product_id"]
    min_fecha = row["min"]
    max_fecha = row["max"]
    periodos_validos = pd.date_range(start=min_fecha, end=max_fecha, freq="MS")
    es_nuevo = min_fecha >= fecha_limite_nuevos  # solo si el producto es nuevo a partir de 2017-02
    
    for p in periodos_validos:
        # if producto in productos_vitales:
        #     combinaciones_producto_periodo.append((p, producto))
        #     continue
        # # Excluir primeros 3 meses si es nuevo (a partir de 2017-02)
        # if es_nuevo and (p < min_fecha + pd.DateOffset(months=3)):
        #     continue
        # # Excluir últimos 3 meses del producto
        # if p > max_fecha - pd.DateOffset(months=3):
        #     continue
        combinaciones_producto_periodo.append((p, producto))

df_producto_periodo = pd.DataFrame(combinaciones_producto_periodo, columns=["periodo_dt", "product_id"])

# Paso 5: Generar combinaciones de todos los clientes con (periodo, producto)
combinaciones = []
for _, row in df_producto_periodo.iterrows():
    periodo = row["periodo_dt"]
    producto = row["product_id"]
    for cliente in todos_los_clientes:
        # if producto in df[df["customer_id"] == cliente]["product_id"].unique(): ###### <------ ESTO TARDA 3 AÑOS
        combinaciones.append((periodo, producto, cliente)) 

df_completo = pd.DataFrame(combinaciones, columns=["periodo_dt", "product_id", "customer_id"])

# Paso 6: Unir con toneladas efectivas
df_merge = df_completo.merge(df[["periodo_dt", "product_id", "customer_id", "tn"]],
                             on=["periodo_dt", "product_id", "customer_id"],
                             how="left")
df_merge["tn"] = df_merge["tn"].fillna(0)

# Paso 7: Recuperar periodo AAAAMM si lo necesitás
df_merge["periodo"] = df_merge["periodo_dt"].dt.strftime("%Y%m").astype(int)

# Resultado final
df_final = df_merge[["periodo", "product_id", "customer_id", "tn"]]

# Vista previa
print(df_final.head())

Elimino los clientes que murieron

In [ ]:
df_final["periodo_dt"] = pd.to_datetime(df_final["periodo"].astype(str), format="%Y%m")

muerte_cliente = df.groupby("customer_id")["periodo_dt"].agg(["max"]).reset_index()

# Renombrar columna max a muerte_cliente_dt
muerte_cliente = muerte_cliente.rename(columns={'max': 'muerte_cliente_dt'})

# Unir con df_final para traer fecha de muerte del cliente
df_final = df_final.merge(muerte_cliente, on='customer_id', how='left')

# Filtrar filas donde periodo_dt > muerte_cliente_dt
df_final = df_final[df_final['periodo_dt'] <= df_final['muerte_cliente_dt']]

# Opcional: eliminar columna auxiliar
# df_final = df_final.drop(columns=['muerte_cliente_dt'])

print(f"✅ Dataset filtrado con {len(df_final):,} filas.")


✅ Dataset filtrado con 17,208,299 filas.


Quito los productos nuevos a partir de 2019-11

In [ ]:
# Obtener lista de productos muy nuevos
nacimiento_producto = df.groupby("product_id")["periodo_dt"].agg(["min"]).reset_index()
nacimiento_producto = nacimiento_producto.rename(columns={'min': 'nacimiento_producto'})
productos_muy_nuevos = nacimiento_producto[nacimiento_producto['nacimiento_producto'] > '2019-10-01']['product_id']


df_final_copy = df_final.copy()



# Filtrar df_final para eliminar esos productos
df_final_copy = df_final_copy[~df_final_copy['product_id'].isin(productos_muy_nuevos)]

print(f"✅ Dataset filtrado: {len(df_final_copy):,} filas restantes.")


Verifico que los productos que voy a predecir esten en 2019-12

In [ ]:
mask = df_final_copy['periodo'] == 201912
df_final_copy[mask]['product_id'].nunique()

productitos = pd.read_csv("../data/raw/product_id_apredecir201912.csv")
productitos['product_id']


# Obtener los conjuntos
productos_en_df = set(df_final_copy[mask]['product_id'].unique())
productos_de_productitos = set(productitos['product_id'].unique())

# Verificar si todos están incluidos
estan_todos = productos_de_productitos.issubset(productos_en_df)

print(f"✅ Todos los productos de 'productitos' están en df_final_copy[mask]: {estan_todos}")

# Si querés ver cuáles faltan
productos_faltantes = productos_de_productitos - productos_en_df
print(f"🔍 Productos faltantes: {productos_faltantes}")


✅ Todos los productos de 'productitos' están en df_final_copy[mask]: True
🔍 Productos faltantes: set()


Verifico si hay duplicados

In [ ]:
duplicados = df_final_copy.duplicated(subset=['periodo', 'product_id', 'customer_id'], keep=False)

# Mostrar si hay duplicados
hay_duplicados = duplicados.any()
print(f"✅ Hay duplicados: {hay_duplicados}")

# Si querés ver las filas duplicadas
if hay_duplicados:
    filas_duplicadas = df_final_copy[duplicados]
    print(f"🔍 Total de filas duplicadas: {len(filas_duplicadas)}")
    print(filas_duplicadas.head())


✅ Hay duplicados: False


Elimino clientes que no habian nacido

In [ ]:
# Convertir 'periodo' a datetime (si no lo hiciste antes)
df_final_copy = df_final.copy()

df_final_copy["periodo_dt"] = pd.to_datetime(df_final_copy["periodo"].astype(str), format="%Y%m")

# Calcular nacimiento del cliente (primer compra de cualquier producto)
nacimiento_cliente = df.groupby("customer_id")["periodo_dt"].min().reset_index().rename(columns={'periodo_dt': 'nacimiento_cliente'})

# Unir con df_final_copy para traer fecha de nacimiento del cliente
df_final_copy = df_final_copy.merge(nacimiento_cliente, on='customer_id', how='left')

# Filtrar filas donde periodo_dt < nacimiento_cliente
df_final_copy = df_final_copy[df_final_copy['periodo_dt'] >= df_final_copy['nacimiento_cliente']]

# Opcional: eliminar columna auxiliar
# df_final_copy = df_final_copy.drop(columns=['nacimiento_cliente'])

print(f"✅ Dataset final filtrado: {len(df_final_copy):,} filas restantes.")


✅ Dataset final filtrado: 15,563,113 filas restantes.


In [ ]:
del df_final_copy
gc.collect()

5926